In [1]:
from __future__ import absolute_import, division, print_function

#!pip install -q tensorflow-gpu==2.0.0-alpha0
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [3]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, 
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [4]:
tokenizer = info.features['text'].encoder

In [5]:
print("Vovaulary size : {}".format(tokenizer.vocab_size))

Vovaulary size : 8185


In [6]:
sample_string = 'TensorFlow is cool.'

In [7]:
tokenized_string = tokenizer.encode(sample_string)
print("Tokenized string is {}".format(tokenized_string))

Tokenized string is [6307, 2327, 4043, 4265, 9, 2724, 7975]


In [8]:
original_string = tokenizer.decode(tokenized_string)
print("The original string is: {}".format(original_string))

The original string is: TensorFlow is cool.


In [9]:
assert original_string == sample_string

In [10]:
for ts in tokenized_string:
    print("{} -------------> {}".format(ts, tokenizer.decode([ts])))

6307 -------------> Ten
2327 -------------> sor
4043 -------------> Fl
4265 -------------> ow 
9 -------------> is 
2724 -------------> cool
7975 -------------> .


In [11]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [12]:

train_dataset = train_dataset.padded_batch(BATCH_SIZE,(tf.TensorShape([None]), tf.TensorShape([])))

test_dataset = test_dataset.padded_batch(BATCH_SIZE, (tf.TensorShape([None]), tf.TensorShape([])))

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

W0318 09:33:35.824431 140216000464704 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f86145189b0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0318 09:33:35.882182 140216000464704 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f861a9b3f98>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [14]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
history = model.fit(train_dataset, epochs=10, 
                    validation_data=test_dataset)

Epoch 1/10
391/391 [==============================] - 72s 185ms/step - loss: 0.6073 - accuracy: 0.6622 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
391/391 [==============================] - 70s 180ms/step - loss: 0.5936 - accuracy: 0.6856 - val_loss: 0.5891 - val_accuracy: 0.7034
Epoch 3/10
391/391 [==============================] - 71s 181ms/step - loss: 0.4814 - accuracy: 0.7721 - val_loss: 0.4895 - val_accuracy: 0.7856
Epoch 4/10
391/391 [==============================] - 70s 178ms/step - loss: 0.3330 - accuracy: 0.8628 - val_loss: 0.3760 - val_accuracy: 0.8346
Epoch 5/10
391/391 [==============================] - 73s 187ms/step - loss: 0.2563 - accuracy: 0.9018 - val_loss: 0.3487 - val_accuracy: 0.8597
Epoch 6/10
391/391 [==============================] - 71s 182ms/step - loss: 0.2066 - accuracy: 0.9249 - val_loss: 0.3456 - val_accuracy: 0.8621
Epoch 7/10
391/391 [==============================] - 69s 177ms/step - loss: 0.1871 - accuracy: 0.9317 - val_loss: 0.3709 

In [16]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

    391/Unknown - 20s 52ms/step - loss: 0.4602 - accuracy: 0.8537Test Loss: 0.4602208761569789
Test Accuracy: 0.8537200093269348


In [17]:
def pad_to_size(vec, size):
    zeros = [0]*(size - len(vec))
    vec.extend(zeros)
    return vec

In [18]:
def sample_predict(sentance, pad):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-18-ee2dde63d1c0>, line 2)